## Fruits and Veggies - Nodes

In [1]:
from src.encoding.configs_and_constants import VSAModel

vsa = VSAModel.HRR
D = 108 * 108  # 96 didn't give 100%
fruits = vsa.tensor_class.random(1, D)
print(isfruits.type())

torch.FloatTensor


In [ ]:
import torch
import torchhd
from src.encoding.configs_and_constants import VSAModel
from src.utils.utils import cartesian_bind_tensor, TupleIndexer
from collections import defaultdict

vsa = VSAModel.HRR
D = 108 * 108  # 96 didn't give 100%
fruits = torchhd.random(28, D, vsa=vsa.value)
veggies = torchhd.random(5, D, vsa=vsa.value)
node_codebook = cartesian_bind_tensor([fruits, veggies])
node_indexer = TupleIndexer([len(fruits), len(veggies)])
print(fruits.t)

# Length 33
basket = [
    tuple(t)
    for t in [
        [0, 1 - 1],
        [0, 2 - 1],
        [0, 2 - 1],
        [0, 2 - 1],
        [0, 2 - 1],
        [0, 2 - 1],
        [0, 3 - 1],
        [2, 2 - 1],
        [2, 3 - 1],
        [0, 3 - 1],
        [2, 1 - 1],
        [0, 3 - 1],
        [0, 2 - 1],
        [0, 3 - 1],
        [0, 3 - 1],
        [0, 1 - 1],
        [2, 3 - 1],
        [0, 3 - 1],
        [0, 2 - 1],
        [0, 2 - 1],
        [0, 3 - 1],
        [0, 1 - 1],
        [0, 2 - 1],
        [0, 3 - 1],
        [0, 1 - 1],
        [0, 3 - 1],
        [0, 1 - 1],
        [0, 2 - 1],
        [0, 3 - 1],
        [1, 1 - 1],
        [2, 2 - 1],
        [0, 3 - 1],
        [5, 2 - 1],
    ]
]
print(basket)
print(len(basket))
print(f"Unique Nodes -> {len(set(basket))=}")
nodes_ground_truth = node_indexer.get_idxs(basket)
print(f"{sorted(set(nodes_ground_truth))=}")

nodes = [node_codebook[node_indexer.get_idx(t)] for t in basket]
embedding_0 = torchhd.multiset(torch.stack(nodes, dim=0))


d = torchhd.dot(embedding_0, node_codebook)
sim_node  = torch.round(d).int().clamp(min=0)
## This should print 3, 0, 1, 0, 1, 0, 3, .... so many time as there items.
## And it does, so it should work
print(sim_node)
print(sim_node.sum().item())



## Fruits and Veggies -- Edges

In [ ]:

## Now level 1
## Length 72
edge_indexes = [
    (0, 1),
    (1, 0),
    (1, 2),
    (2, 1),
    (2, 3),
    (3, 2),
    (3, 4),
    (4, 3),
    (4, 5),
    (5, 4),
    (5, 6),
    (6, 5),
    (6, 7),
    (6, 32),
    (7, 6),
    (7, 8),
    (8, 7),
    (8, 9),
    (8, 31),
    (9, 8),
    (9, 10),
    (9, 11),
    (10, 9),
    (11, 9),
    (11, 12),
    (11, 28),
    (12, 11),
    (12, 13),
    (13, 12),
    (13, 14),
    (13, 27),
    (14, 13),
    (14, 15),
    (14, 16),
    (15, 14),
    (16, 14),
    (16, 17),
    (16, 25),
    (17, 16),
    (17, 18),
    (17, 23),
    (18, 17),
    (18, 19),
    (19, 18),
    (19, 20),
    (20, 19),
    (20, 21),
    (20, 22),
    (21, 20),
    (22, 20),
    (22, 23),
    (23, 17),
    (23, 22),
    (23, 24),
    (24, 23),
    (25, 16),
    (25, 26),
    (25, 27),
    (26, 25),
    (27, 13),
    (27, 25),
    (28, 11),
    (28, 29),
    (28, 30),
    (29, 28),
    (30, 28),
    (30, 31),
    (31, 8),
    (31, 30),
    (31, 32),
    (32, 6),
    (32, 31),
]

print(f"{len(edge_indexes)=}")

edge_dict = defaultdict(list)
for src, dst in edge_indexes:
    edge_dict[src].append(dst)
print(edge_dict)

edge_term_bindings = [
    torchhd.bind(
        node_codebook[node_indexer.get_idx(basket[dst])],
        torchhd.multiset(torch.stack([node_codebook[node_indexer.get_idx(basket[d])] for d in dsts], dim=0)),
    )
    for src, dsts in edge_dict.items()
]
edge_terms = torchhd.multiset(torch.stack(edge_term_bindings, dim=0))
embedding_1 = torchhd.multiset(torch.stack([embedding_0, edge_terms], dim=0))


In [ ]:
edge_codebook = cartesian_bind_tensor([node_codebook, node_codebook])

In [ ]:
## Let's optimize the edge decoder
from src.utils.utils import TupleIndexer


decoded_node_idx = sim_node.nonzero(as_tuple=True)[0].tolist()
print(f"{decoded_node_idx=}")
print(f"{len(decoded_node_idx)=}")

upper = [
    (decoded_node_idx[i], decoded_node_idx[j])
    for i in range(len(decoded_node_idx))
    for j in range(i, len(decoded_node_idx))
]
print(f"{upper=}")

lower = [(v, u) for u, v in upper]
print(f"{lower=}")

edge_indexer = TupleIndexer([node_codebook.shape[0], node_codebook.shape[0]])
print(f"{edge_indexer.get_sizes()=}")

upper_idxs = edge_indexer.get_idxs(upper)
print(f"{upper_idxs=}")
print(f"{len(upper_idxs)=}")

lower_idxs = edge_indexer.get_idxs(lower)
print(f"{lower_idxs=}")
print(f"{len(lower_idxs)=}")

upper_codebook = edge_codebook[upper_idxs]
print(f"{upper_codebook.shape=}")

lower_codebook = edge_codebook[lower_idxs]
print(f"{lower_codebook.shape=}")

In [ ]:
sim_upper = torchhd.dot(edge_terms, upper_codebook).clamp(min=0.0)
sim_lower = torchhd.dot(edge_terms, lower_codebook).clamp(min=0.0)
sim_enhanced = (sim_upper + sim_lower).clamp(min=0.0)
print(f"{sim_enhanced=}")
print(f"{sim_enhanced.shape=}")
print(f"{sim_enhanced.nonzero().shape=}")

decoded_edges = []
for i in range(sim_enhanced.shape[0]):
    if sim_enhanced[i].item() > 0: # Edge exists
        decoded_edges.append(upper[i])
        decoded_edges.append(lower[i])
print(f"{decoded_edges=}")
print(f"{len(decoded_edges)=}")

real_edges = [(node_indexer.get_idx(basket[u]), node_indexer.get_idx(basket[v]))for u, v in edge_indexes]
print(f"{real_edges=}")
print(f"{len(real_edges)=}")